<a href="https://colab.research.google.com/github/soujanya-vattikolla/MongoDB-basics/blob/main/Chapter4%20AdvancedCRUDOperations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Query Operators**- It provides the additional ways to locate data within the database.<br>
**$** - has multiple uses.
* precedes MQL operators
* precedes aggregation pipeline stages
* allows access to field values

**Comparison Operators**

In [ ]:
# $eq = EQual to 
# $ne = Not Equal to

# $gt > Greater Than
# $lt < Less Than

# $gte >= Greater Than or Equal To
# $lt <= Less Than or Equal To

# {<field>: {<operator>: <value>}}


When the comparator operator is not specified then $eq is default operator.

In [ ]:
# Connect to the Atlas cluster:
# mongo "mongodb+srv://<username>:<password>@<cluster>.mongodb.net/admin"

# use sample_training

# Find all documents where the tripduration was less than or equal to 70 seconds and the usertype was not Subscriber:

db.trips.find({ "tripduration": { "$lte" : 70 },
                "usertype": { "$ne": "Subscriber" } }).pretty()

# Find all documents where the tripduration was less than or equal to 70 seconds and the usertype was Customer using a redundant equality operator:

db.trips.find({ "tripduration": { "$lte" : 70 },
                "usertype": { "$eq": "Customer" }}).pretty()

# Find all documents where the tripduration was less than or equal to 70 seconds and the usertype was Customer using the implicit equality operator:

db.trips.find({ "tripduration": { "$lte" : 70 },
                "usertype": "Customer" }).pretty()


**Problem:**

How many documents in the sample_training.zips collection have fewer than 1000 people listed in the pop field? <br>

db.zips.find({ "pop": { "$lt": 1000 }}).count()

**Question**: What is the difference between the number of people born in 1998 and the number of people born after 1998 in the sample_training.trips collection?<br>
db.trips.find({"birth year":1998}).count()  <br>
db.trips.find({"birth year":{"$gt":1998}}).count() 

In [ ]:
# Question: 

Using the sample_training.routes collection find out which of the following statements will return all routes that have at least one stop in them?

db.routes.find({ "stops": { "$gt": 0 }}).pretty()

The given query looks for strict equality where the stops field has to be greater than zero, thus excluding all zero stops.

db.routes.find({ "stops": { "$ne": 0 }}).pretty()

This query will also work, given that there are no non-negative or non- numeric values in this collection. It returns all documents where the stops field is not equal to 0.

In [ ]:
Logic operators

1. $and : Match all of the specified query clauses.
2. $or : At least one of the query clauses is matched.
3. $nor: Fail to match both given clauses.
4. $not: Negates the query requirement.

In [ ]:
1. $and : Match all of the specified query clauses.
2. $or : At least one of the query clauses is matched.
3. $nor: Fail to match both given clauses.
has the following statement:

{<operator> : [ {statement1}, {statement2},...]}

4. $not: 
{$not : [ {statement}}

$and is used as the **default operator** when an operator is not specified.

**Implicit $and** 

In [ ]:
Example: Find which student ids are (>25) and (<100) in the sample_training.grades collection

{"student_id": {"$gt":25, "$lt":100}}

**Explicit $and**

In [ ]:
Example: Using the routes collection find out how many CR2 and A81 airplanes come through the KZN airport?

{"$and": [ {"$or": [{dst_airport : "KZN"}, {src_airport: "KZN"}]}, { {"$or": [{airplane : "CR2"}, {airplane: "A81"}]}}}

In [ ]:
Question: 

How many businesses in the sample_training.inspections dataset have the inspection result "Out of Business" and belong to the "Home Improvement Contractor - 100" sector?

{"$and": [ { result : "Out of Business"}, {sector : "Home Improvement Contractor - 100"}]}

result: 4


In [ ]:
Problem:

Which is the most succinct query to return all documents from the sample_training.inspections collection where the inspection date is either "Feb 20 2015", or "Feb 21 2015" and the company is not part of the "Cigarette Retail Dealer - 127" sector?

db.inspections.find({"$or": [ { "date": "Feb 20 2015"}, 
                             { "date": "Feb 21 2015"}], 
                     "sector": {"$ne":"Cigaratte Retail Dealer-127"}}).pretty()

In [ ]:
How many zips in the sample_training.zips dataset are neither over-populated nor under-populated?
In this case, we consider population of more than 1,000,000 to be over- populated and less than 5,000 to be under-populated.

db.zips.find({ "pop": { "$gte": 5000, "$lte": 1000000 }}).count()

or 

db.zips.find({ "$nor": [ { "pop": { "$lt":5000 } },
             { "pop": { "$gt": 1000000 } } ] } ).count()

In [ ]:
How many companies in the sample_training.companies dataset were either founded in 2004
[and] either have the social category_code [or] web category_code,
[or] were founded in the month of October
[and] also either have the social category_code [or] web category_code?


db.companies.find({ "$and": [
                        { "$or": [ { "founded_year": 2004 },
                                   { "founded_month": 10 } ] },
                        { "$or": [ { "category_code": "web" },
                                   { "category_code": "social" }]}]}).count()


**Expressive Query Operator**

In [ ]:
$expr:
1. It allows the use of aggregation expressions within the query language.
2. $expr: { <expression> }
3. $expr also allows us to use variables and conditional statements.

**$** denotes the use of an operator and addresses the field value.

**Aggregation Syntax**

In [ ]:
{ <operator> : { <field>, <value> } }

In [ ]:
# Examples:

1. Find all documents where the trip started and ended at the same station:

db.trips.find({ "$expr": { "$eq": [ "$end station id", "$start station id"] } }).count()

2. Find all documents where the trip lasted longer than 1200 seconds, and started and ended at the same station:

db.trips.find({ "$expr": { "$and": [ { "$gt": [ "$tripduration", 1200 ]},
                         { "$eq": [ "$end station id", "$start station id" ]}
                         ]}}).count()


In [ ]:
# Problem:


Which of the following statements will find all the companies that have more employees than the year in which they were founded?

1. db.companies.find(
    { "$expr": { "$lt": [ "$founded_year", "$number_of_employees" ] } }).count()

  or

2. db.companies.find(
    { "$expr": { "$gt": [ "$number_of_employees", "$founded_year" ]} }).count()


In [ ]:
Question:
How many companies in the sample_training.companies collection have the same permalink as their twitter_username?

db.companies.find({ "$expr": { "$eq": [ "$permalink", "$twitter_username" ] }}).count()

**Array Operators**

$push, 
* To add an element to an array 
* Turn a field into an array field if it was previously a different type of value.

**$size** 
* The array operator will return**all documents where the specified array field is exactly the given length.**


**$all** 
* The array operator will return a cursor with all documents in which the specified array field contains all the given elements, **regardless of their order in the array.**


* When **querying an array field with an array match**, MongoDB will look for an **exact array match**, unless specified otherwise.

* When **querying an array field with a single element**, MongoDB will return all documents where the **specified array field contains this given element**.

In [ ]:
# Example:

Find all documents with exactly 20 amenities which include all the amenities listed in the query array:

db.listingsAndReviews.find({ "amenities": {
                                  "$size": 20,
                                  "$all": [ "Internet", "Wifi",  "Kitchen",
                                           "Heating", "Family/kid friendly",
                                           "Washer", "Dryer", "Essentials",
                                           "Shampoo", "Hangers",
                                           "Hair dryer", "Iron",
                                           "Laptop friendly workspace" ]
                                         }
                            }).pretty()

In [ ]:
# Question:

What is the name of the listing in the sample_airbnb.listingsAndReviews dataset that accommodates more than 6 people and has exactly 50 reviews?

db.listingsAndReviews.find({ "reviews": { "$size":50 },
                             "accommodates": { "$gt":6 }})



In [ ]:
#  Question

Using the sample_airbnb.listingsAndReviews collection find out how many documents have the "property_type" "House", and include "Changing table" as one of the "amenities"?

db.listingsAndReviews.find({ "property_type": "House",
                             "amenities": "Changing table" }).count()

or

db.listingsAndReviews.find({ "property_type" :"House", "amenities" : {"$all": ["Changing table"] } }).count()


In [ ]:
# Question:

Which of the following queries will return all listings that have "Free parking on premises", "Air conditioning", and "Wifi" as part of their amenities, and have at least 2 bedrooms in the sample_airbnb.listingsAndReviews collection?

db.listingsAndReviews.find(
  { "amenities":
     { "$all": [ "Free parking on premises", "Wifi", "Air conditioning" ] },
    "bedrooms": { "$gte":  2 } }).pretty()
    

**Array Operators and Projection**

In [ ]:
 # Projection syntax

 db.<collection>.find( {<query>}, {<projection>})

 1 - include the field
 0 - exclude the field
 Use only 1s and 0s

**elemMatch**

In [ ]:
{<field> : { "$elemMatch": { <field> : <value> }}}

elemMatch matches documents that contain an array field with at least one element that matches all the specified query criteria, or 
projects only the array elements with at least one element that matches the specified criteria.

In [ ]:
# Find all documents with exactly 20 amenities which include all the amenities listed in the query array, and display their price and address:

db.listingsAndReviews.find({ "amenities":
        { "$size": 20, "$all": [ "Internet", "Wifi",  "Kitchen", "Heating",
                                 "Family/kid friendly", "Washer", "Dryer",
                                 "Essentials", "Shampoo", "Hangers",
                                 "Hair dryer", "Iron",
                                 "Laptop friendly workspace" ] } },
                            {"price": 1, "address": 1}).pretty()


In [ ]:
# Find all documents that have Wifi as one of the amenities only include price and address in the resulting cursor:

db.listingsAndReviews.find({ "amenities": "Wifi" },
                           { "price": 1, "address": 1, "_id": 0 }).pretty()


In [ ]:
# Find all documents where the student in class 431 received a grade higher than 85 for any type of assignment:

db.grades.find({ "class_id": 431 },
               { "scores": { "$elemMatch": { "score": { "$gt": 85 } } }
             }).pretty()

In [ ]:
# Find all documents where the student had an extra credit score:

db.grades.find({ "scores": { "$elemMatch": { "type": "extra credit" } }
               }).pretty()

In [ ]:
Question:

How many companies in the sample_training.companies collection have offices in the city of Seattle?

db.companies.find({ "offices": { "$elemMatch": { "city": "Seattle" } }
                  }).count()

In [ ]:
Question

Which of the following queries will return only the names of companies from the sample_training.companies collection that had exactly 8 funding rounds?

db.companies.find({ "funding_rounds": { "$size": 8 } },
                  { "name": 1, "_id": 0 })

**Array Operators and Sub-Documents**

MQL uses **dot notation** to specify the address of nested elements in a document.<br>
To specify the position of the element in the array.


In [ ]:
# Examples:

use sample_training

db.trips.findOne({ "start station location.type": "Point" })

db.companies.find({ "relationships.0.person.last_name": "Zuckerberg" },
                  { "name": 1 }).pretty()

db.companies.find({ "relationships.0.person.first_name": "Mark",
                    "relationships.0.title": { "$regex": "CEO" } },
                  { "name": 1 }).count()


db.companies.find({ "relationships.0.person.first_name": "Mark",
                    "relationships.0.title": {"$regex": "CEO" } },
                  { "name": 1 }).pretty()

db.companies.find({ "relationships":
                      { "$elemMatch": { "is_past": true,
                                        "person.first_name": "Mark" } } },
                  { "name": 1 }).pretty()

db.companies.find({ "relationships":
                      { "$elemMatch": { "is_past": true,
                                        "person.first_name": "Mark" } } },
                  { "name": 1 }).count()

In [ ]:
Question

How many trips in the sample_training.trips collection started at stations that are to the west of the -74 longitude coordinate?

db.trips.find({ "start station location.coordinates.0": { "$lt": -74 }}).count() 

or 

db.trips.find({ "start station location.coordinates": { "$lt": -74 }}).count() 

In [ ]:
Question:

How many inspections from the sample_training.inspections collection were conducted in the city of NEW YORK?


db.inspections.find({ "address.city": "NEW YORK" }).count()

In [ ]:
Problem:

Which of the following queries will return the names and addresses of all listings from the sample_airbnb.listingsAndReviews collection where the first amenity in the list is "Internet"?

db.listingsAndReviews.find({ "amenities.0": "Internet" },
                           { "name": 1, "address": 1 }).pretty()
                           